In [3]:
from kafka import KafkaConsumer,TopicPartition

import sys
sys.path.append("../..")
import utils

from opentracing_instrumentation.request_context import get_current_span, span_in_context

def consumer_single_collection(topic):
    consumer = KafkaConsumer(bootstrap_servers='localhost:9092')
    tp = TopicPartition(topic,0)
    consumer.assign([tp])
    consumer.seek_to_end(tp)
    lastOffset = consumer.position(tp)
    print(lastOffset)
    consumer.seek_to_beginning(tp) 
    #end
    list_data=[]
    with tracer.start_span('collect_data',child_of=get_current_span()) as span:
        span.set_tag('kafka','collect_data_jour_1')
        with span_in_context(span):
            for message in consumer:
                msg = message.value.decode()
                mongodata={
                    'timestamp':msg.split(";")[0].split(":",1)[1],
                    'msg':msg.split(";",1)[1]
                }
                list_data.append(mongodata)
                if message.offset == lastOffset - 1:
                    print(message.offset)
                    break
            consumer.close()
            return list_data

def insert_bulk(list_data,collection):
    with tracer.start_span('insert_bulk',child_of=get_current_span()) as span:
        span.set_tag('mongodb','operation:insert_many')
        with span_in_context(span):
            try :
                collection.insert_many(list_data)
                return "sucess"
            except : 
                return "error"
            
def test_insert_bulk(collection,topic):
    with tracer.start_span('test_bulk_insertion') as span:
        span.set_tag('mongodb','insertion_test')
        with span_in_context(span):
            data=consumer_single_collection(topic)
            result=insert_bulk(data,collection)
            return result
            
            
if __name__=="__main__":
    client = utils.mongodb_connect("localhost", 28018)
    topic="eolienne_jour_7"
    col=client.test['test_bulk_eolienne_insert']
    tracer = utils.init_tracer('mongodb_test_eolienne_1_jour')
    result = test_insert_bulk(col,topic)
    print(result)

Initializing Jaeger Tracer with UDP reporter
Using selector: EpollSelector
Using sampler ConstSampler(True)
Added sensor with name connections-closed
Added sensor with name connections-created
Added sensor with name select-time
Added sensor with name io-time
Initiating connection to node bootstrap-0 at localhost:9092
Added sensor with name bytes-sent-received
Added sensor with name bytes-sent
Added sensor with name bytes-received
Added sensor with name request-latency
Added sensor with name node-bootstrap-0.bytes-sent
Added sensor with name node-bootstrap-0.bytes-received
Added sensor with name node-bootstrap-0.latency
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <disconnected> [unspecified None]>: creating new socket
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <disconnected> [IPv6 ('::1', 9092, 0, 0)]>: setting socket option (6, 1, 1)
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9

Broker version identified as 1.0.0
Set configuration api_version=(1, 0, 0) to skip auto check_version requests on startup
Added sensor with name bytes-fetched
Added sensor with name records-fetched
Added sensor with name fetch-latency
Added sensor with name records-lag
Added sensor with name fetch-throttle-time
Added sensor with name heartbeat-latency
Added sensor with name join-latency
Added sensor with name sync-latency
group_id is None: disabling auto-commit.
Added sensor with name commit-latency
Seeking to end of partition TopicPartition(topic='eolienne_jour_7', partition=0)
Resetting offset for partition TopicPartition(topic='eolienne_jour_7', partition=0) to latest offset.
Partition TopicPartition(topic='eolienne_jour_7', partition=0) is unknown for fetching offset, wait for metadata refresh
Sending metadata request MetadataRequest_v1(topics=['eolienne_jour_7']) to node bootstrap-0
Sending request MetadataRequest_v1(topics=['eolienne_jour_7'])
<BrokerConnection node_id=bootstrap-

Trying to connect to MongoDB server: localhost on port: 28018
error : create new tracer...


Sending request OffsetRequest_v1(replica_id=-1, topics=[(topic='eolienne_jour_7', partitions=[(partition=0, timestamp=-1)])])
<BrokerConnection node_id=0 host=irlin328206:9092 <connected> [IPv6 ('fe80::9a90:96ff:fead:e31d', 9092, 0, 2)]> Request 1: OffsetRequest_v1(replica_id=-1, topics=[(topic='eolienne_jour_7', partitions=[(partition=0, timestamp=-1)])])
Received correlation id: 1
Processing response OffsetResponse_v1
<BrokerConnection node_id=0 host=irlin328206:9092 <connected> [IPv6 ('fe80::9a90:96ff:fead:e31d', 9092, 0, 2)]> Response 1 (2.5665760040283203 ms): OffsetResponse_v1(topics=[(topic='eolienne_jour_7', partitions=[(partition=0, error_code=0, timestamp=-1, offset=48776)])])
Handling ListOffsetResponse response for TopicPartition(topic='eolienne_jour_7', partition=0). Fetched offset 48776, timestamp -1
Seeking to beginning of partition TopicPartition(topic='eolienne_jour_7', partition=0)
Resetting offset for partition TopicPartition(topic='eolienne_jour_7', partition=0) to 

48776


...

Adding fetched record for partition TopicPartition(topic='eolienne_jour_7', partition=0) with offset 26591 to buffered record list
Advance position for partition TopicPartition(topic='eolienne_jour_7', partition=0) from 30591 to 31034 (last message batch location plus one) to correct for deleted compacted messages
Adding fetch request for partition TopicPartition(topic='eolienne_jour_7', partition=0) at offset 31034
Sending FetchRequest to node 0
Sending request FetchRequest_v4(replica_id=-1, max_wait_time=500, min_bytes=1, max_bytes=52428800, isolation_level=0, topics=[(topic='eolienne_jour_7', partitions=[(partition=0, offset=31034, max_bytes=1048576)])])
<BrokerConnection node_id=0 host=irlin328206:9092 <connected> [IPv6 ('fe80::9a90:96ff:fead:e31d', 9092, 0, 2)]> Request 10: FetchRequest_v4(replica_id=-1, max_wait_time=500, min_bytes=1, max_bytes=52428800, isolation_level=0, topics=[(topic='eolienne_jour_7', partitions=[(partition=0, offset=31034, max_bytes=1048576)])])
Received co

Adding fetched record for partition TopicPartition(topic='eolienne_jour_7', partition=0) with offset 48416 to buffered record list
Closing the KafkaConsumer.
<BrokerConnection node_id=0 host=irlin328206:9092 <connected> [IPv6 ('fe80::9a90:96ff:fead:e31d', 9092, 0, 2)]>: Closing connection. 
The KafkaConsumer has closed.
Reporting span 6a9f3a337d82716a:5e762ed90320bfab:a918256b571a61cb:1 mongodb_test_eolienne_1_jour.collect_data


48775


Reporting span 6a9f3a337d82716a:44bf030b68c94894:a918256b571a61cb:1 mongodb_test_eolienne_1_jour.insert_bulk
Reporting span 6a9f3a337d82716a:a918256b571a61cb:0:1 mongodb_test_eolienne_1_jour.test_bulk_insertion


sucess
